In [577]:
import numpy as np
import pandas as pd
import math
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

def printVar (varname, var):

    print(varname)
    print(var)

def saveArray (name, array):

    pd.DataFrame(array).to_csv("savedVariables/" + name + ".csv", index=False, header=None)


In [578]:
# Solve the 2D Navier-Cauchy equation using a segregated finite volume method

# Define mesh geometry and boundary conditions

Lx = .01 #width of domain
Ly = .01 #height of domain

fac = 1

# tr_left_x = -1e-6*fac      #u boundary condition at the left boundary
# tr_left_y =  1e-6*0     #v boundary condition at the left boundary

# tr_right_x = +1e-6*fac    #u boundary condition at the right boundary
# tr_right_y = 1e-6*0     #v boundary condition at the right boundary

# tr_top_x = 1e-6*0       #u boundary condition at the top boundary
# tr_top_y = +1e-6*fac       #v boundary condition at the top boundary

# tr_bottom_x = -1e-6*0    #u boundary condition at the bottom boundary 
# tr_bottom_y = -1e-6*fac    #v boundary condition at the bottom boundary

nx = 8          #number of control volumes in the x-direction
ny = 8          #number of control volumes in the y-direction

u_bottom = -1e-6
v_bottom = -1e-6

u_top = 1e-6
v_top = 1e-6

u_left = -1e-6
v_left = -1e-6

u_right = 1e-6
v_right = 1e-6

In [579]:
class edge_U:

    def __init__(self, edges, xy):

        if xy == "x":
            if edges[0] == "b":
                self.BC = u_bottom
            if edges[0] == "t":
                self.BC = u_top
            if edges[0] == "l":
                self.BC = u_left
            if edges[0] == "r":
                self.BC = u_right

        if xy == "y":
            if edges[0] == "b":
                self.BC = v_bottom
            if edges[0] == "t":
                self.BC = v_top
            if edges[0] == "l":
                self.BC = v_left
            if edges[0] == "r":
                self.BC = v_right

edge_U(["b"], "x").BC

-1e-06

In [580]:
sys.path.append("/home/scottlevie/pythonsolver/tractionBCs/Classes/")

from index_and_direction import dim

# dim().nx = 9
# dim().ny = 9

# print(dim().nx)

In [581]:
print(dim().nx)

8


In [582]:
from index_and_direction import index
from index_and_direction import boundary_point_index
from index_and_direction import cell_index
from index_and_direction import displacement

In [583]:
# Define material properties

rho = 8050         #density of steel in kg/m^3
mu = 78*10e9       #shear modulus in Pa
K = 1.6*10e11      #bulk modulus in Pa

Lambda = K - (2/3)*mu

#print(Lambda)

In [584]:
## Define required variables

tf = 0.01           #total time in seconds
dx = Lx/nx          #length of each control volume in the x-direction
dy = Ly/ny          #length of each control volume in the y-direction
dt = tf/10          #size of time steps
Sfx = dy            #area vector x component (Area of East and West Faces)
Sfy = dx            #area vector y component (Area of North and South Faces)


In [585]:
## Define  position and time vectors

t = np.array(np.arange(0,tf, dt))        #time vector (could possibly use a list)
x = np.zeros((1,nx+2))   #position vector defining the center points of the control volumes in the x-direction.
x[0,nx+1] = Lx
x[0,1:nx+1] = np.arange(dx/2,Lx,dx)

t = np.array(np.arange(0,tf, dt))        #time vector (could possibly use a list)
y = np.zeros((1,ny+2))   #position vector defining the center points of the control volumes in the y-direction.
y[0,ny+1] = Ly
y[0,1:ny+1] = np.arange(dy/2,Ly,dy)

In [586]:
## Define required displacement matrices and b vector

u_new = np.zeros([(ny+2)*(nx+2),2])         #unknown displacements at t + 1    (Column 1 = x, Column2 = y)
u_old = np.zeros([(ny+2)*(nx+2),2])         #displacement at time t
u_old_old = np.zeros([(ny+2)*(nx+2),2])     #displacement at time t - 1

#rhs vector
b_x = np.zeros([(ny+2)*(nx+2),1])
b_y = np.zeros([(ny+2)*(nx+2),1])

## X equation:

In [587]:
class A(index, displacement):

    val = "value that stays in class without changning with instance"

    def __init__(self, xy):

        if xy == "x":

            self.a_P = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfx)/dx + ((2*mu)*Sfy)/dy
            
            self.a_N = + A.coef(xy, "N", "u")*(Sfy)/dy
            self.a_S = - A.coef(xy, "S", "u")*(Sfy)/dy
            self.a_E = + A.coef(xy, "E", "u")*(Sfx)/dx
            self.a_W = - A.coef(xy, "W", "u")*(Sfx)/dx

        if xy == "y":

            self.a_P = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfy)/dy + ((2*mu)*Sfx)/dx 
            self.a_N = + A.coef(xy, "N", "v")*(Sfy)/dy
            self.a_S = - A.coef(xy, "S", "v")*(Sfy)/dy
            self.a_E = + A.coef(xy, "E", "v")*(Sfx)/dx
            self.a_W = - A.coef(xy, "W", "v")*(Sfx)/dx

    def createMatrix(self):

        A_no_boundary = np.zeros([(ny+2)*(nx+2), (ny+2)*(nx+2)])
        A_no_boundary = csr_matrix(A_no_boundary)  

        for j in np.arange(2, ny):     

            for k in np.arange((j*(nx+2)+2), (j*(nx+2)+nx)):  
                
                A_no_boundary[k,k] = self.a_P
                A_no_boundary[k,index(k).n] = -self.a_N
                A_no_boundary[k,index(k).s] = -self.a_S
                A_no_boundary[k, index(k).e] = -self.a_E
                A_no_boundary[k, index(k).w] = -self.a_W

        return A_no_boundary

    # b temporal term
    def b_temp(u_old, u_old_old, k, xy):

        if xy == "x":
            b_temporal_term = (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy)
        if xy == "y":
            b_temporal_term = (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy)
        
        return b_temporal_term
    
    def b_diff(k, xy, u_previous):

        if xy == "x":
            uv = "v"
        if xy == "y":
            uv = "u"

        b_diffusion = (
                                Sfy*A.coef(xy, "N", uv)*(
                                    (A.corner("NE", uv, u_previous, k) - A.corner("NW", uv, u_previous, k))
                                    /dx)
                                + 
                                Sfy*A.coef(xy, "S", uv)*(
                                    (A.corner("SE", uv, u_previous, k) - A.corner("SW", uv, u_previous, k))
                                    /dx)
                                +
                                Sfx*A.coef(xy, "E", uv)*(
                                    (A.corner("NE", uv, u_previous, k) - A.corner("SE", uv, u_previous, k))
                                    /dy)
                                +
                                Sfx*A.coef(xy, "W", uv)*(
                                    (A.corner("NW", uv, u_previous, k) - A.corner("SW", uv, u_previous, k))
                                    /dy)
        )

        return b_diffusion
    
    def corner(corner_placement, uv, u_previous, k):
        if uv == "u":
            uv_i = 0
        elif uv == "v":
            uv_i = 1

        disp = displacement(k, u_previous, uv_i)

        if corner_placement == "NE":
            return (1/4)*(disp.P + disp.NE + disp.N + disp.E)
        if corner_placement == "SE":
            return (1/4)*(disp.P + disp.SE + disp.S + disp.E)
        if corner_placement == "SW":
            return (1/4)*(disp.P + disp.SW + disp.S + disp.W)
        if corner_placement == "NW":
            return (1/4)*(disp.P + disp.NW + disp.N + disp.W)


    
    def coef(xy, face, uv):
        # could have and statements
        if xy == "x":
            if face == "N":
                if uv == "u":
                    return mu
                if uv == "v":
                    return mu
            if face == "S":
                if uv == "u":
                    return - mu
                if uv == "v":
                    return - mu
            if face == "E":
                if uv == "u":
                    return 2*mu + Lambda
                if uv == "v":
                    return Lambda
            if face == "W":
                if uv == "u":
                    return - (2*mu + Lambda)
                if uv == "v":
                    return - Lambda

        elif xy == "y":
            if face == "N":
                if uv == "u":
                    return Lambda
                if uv == "v":
                    return 2*mu + Lambda
            if face == "S":
                if uv == "u":
                    return - Lambda
                if uv == "v":
                    return - (2*mu + Lambda)
            if face == "E":
                if uv == "u":
                    return mu
                if uv == "v":
                    return mu
            if face == "W":
                if uv == "u":
                    return - mu
                if uv == "v":
                    return - mu

u_previous = u_new

# A.b_diff(25, "x", u_previous)

In [588]:
class boundaryCellTraction(A):

    def __init__(self, edges, xy):
        
        sw_S, sw_N, sw_W, sw_E = 1, 1, 1, 1

        for edge in edges:
            if edge == "b": sw_S = 0
            if edge == "t": sw_N = 0
            if edge == "l": sw_W = 0
            if edge == "r": sw_E = 0
        
        self.a_P = (rho*dx*dy/(dt**2)) + A(xy).a_N*sw_N + A(xy).a_S**sw_S + A(xy).a_E*sw_E + A(xy).a_W*sw_W
        
        self.a_N = A(xy).a_N 
        self.a_S = A(xy).a_S 
        self.a_E = A(xy).a_E 
        self.a_W = A(xy).a_W 

        # self.a_faces = A(xy).a_N*sw_N + A(xy).a_S*sw_S + A(xy).a_E*sw_E + A(xy).a_W*sw_W 

    def b_temp(u_old, u_old_old, k, xy):      
        return A.b_temp(u_old, u_old_old, k, xy)
    
    def b_diff(edges, k, xy, u_previous):

        if xy == "x":
            uv = "v"
        if xy == "y":
            uv = "u"
            
        N_term =(
                    Sfy*A.coef(xy, "N", uv)*(
                    (boundaryCellTraction.corner(edges, "NE", uv, u_previous, k) - boundaryCellTraction.corner(edges, "NW", uv, u_previous, k))
                    /dx)
                )
        S_term =(
                    Sfy*A.coef(xy, "S", uv)*(
                        (boundaryCellTraction.corner(edges, "SE", uv, u_previous, k) - boundaryCellTraction.corner(edges, "SW", uv, u_previous, k))
                        /dx)
                ) 
        E_term =(
                    Sfx*A.coef(xy, "E", uv)*(
                        (boundaryCellTraction.corner(edges, "NE", uv, u_previous, k) - boundaryCellTraction.corner(edges, "SE", uv, u_previous, k))
                        /dy)
                ) 
        W_term =(
                    Sfx*A.coef(xy, "W", uv)*(
                        (boundaryCellTraction.corner(edges, "NW", uv, u_previous, k) - boundaryCellTraction.corner(edges, "SW", uv, u_previous, k))
                        /dy)
                )

        for edge in edges:

            if (edge == "b") & (xy == "x") : S_term =  Sfy*tr_bottom_x  
            if (edge == "b") & (xy == "y") : S_term =  Sfy*tr_bottom_y  
            if (edge == "t") & (xy == "x") : N_term =  Sfy*tr_top_x  
            if (edge == "t") & (xy == "y") : N_term =  Sfy*tr_top_y  
            if (edge == "l") & (xy == "x") : W_term =  Sfx*tr_left_x  
            if (edge == "l") & (xy == "y") : W_term =  Sfx*tr_left_y  
            if (edge == "r") & (xy == "x") : E_term =  Sfx*tr_right_x  
            if (edge == "r") & (xy == "y") : E_term =  Sfx*tr_right_y        

        b_diffusion = (N_term + S_term + E_term + W_term)

        return b_diffusion
    
    def corner(edges, corner_placement, uv, u_previous, k):

        if uv == "u":
            uv_i = 0
        elif uv == "v":
            uv_i = 1

        disp = displacement(k, u_previous, uv_i)

        if corner_placement == "NE":
            corner =  (1/4)*(disp.P + disp.NE + disp.N + disp.E)
        if corner_placement == "SE":
            corner =  (1/4)*(disp.P + disp.SE + disp.S + disp.E)
        if corner_placement == "SW":
            corner =  (1/4)*(disp.P + disp.SW + disp.S + disp.W)
        if corner_placement == "NW":
            corner =  (1/4)*(disp.P + disp.NW + disp.N + disp.W)

        for edge in edges:
            if (edge == "b") & (corner_placement == "SE"):
                corner =  (1/2)*(disp.SE + disp.S)
            if (edge == "b") & (corner_placement == "SW"):
                corner =  (1/2)*(disp.SW + disp.S)

            if (edge == "t") & (corner_placement == "NE"):
                corner =  (1/2)*(disp.NE + disp.N)
            if (edge == "t") & (corner_placement == "NW"):
                corner =  (1/2)*(disp.NW + disp.N)

            if (edge == "l") & (corner_placement == "NW"):
                corner =  (1/2)*(disp.NW + disp.W)
            if (edge == "l") & (corner_placement == "SW"):
                corner =  (1/2)*(disp.SW + disp.W)

            if (edge == "r") & (corner_placement == "NE"):
                corner =  (1/2)*(disp.NE + disp.E)
            if (edge == "r") & (corner_placement == "SE"):
                corner =  (1/2)*(disp.SE + disp.E)
        
        return corner

# boundaryCellTraction.b_diff(["b", "l"], 14, "x", u_previous)
   

In [589]:
class boundaryCellTraction(A):

    def __init__(self, edges, xy):
        
        sw_S, sw_N, sw_W, sw_E = 1, 1, 1, 1

        for edge in edges:
            if edge == "b": sw_S = 0
            if edge == "t": sw_N = 0
            if edge == "l": sw_W = 0
            if edge == "r": sw_E = 0
        
        self.a_P = (rho*dx*dy/(dt**2)) + A(xy).a_N*sw_N + A(xy).a_S*sw_S + A(xy).a_E*sw_E + A(xy).a_W*sw_W
        
        self.a_N = A(xy).a_N 
        self.a_S = A(xy).a_S 
        self.a_E = A(xy).a_E 
        self.a_W = A(xy).a_W 

        # self.a_faces = A(xy).a_N*sw_N + A(xy).a_S*sw_S + A(xy).a_E*sw_E + A(xy).a_W*sw_W 

    def b_temp(u_old, u_old_old, k, xy):      
        return A.b_temp(u_old, u_old_old, k, xy)
    
    def b_diff(edges, k, xy, u_previous):

        if xy == "x":
            uv = "v"
        if xy == "y":
            uv = "u"
            
        N_term =(
                    Sfy*A.coef(xy, "N", uv)*(
                    (boundaryCellTraction.corner(edges, "NE", uv, u_previous, k) - boundaryCellTraction.corner(edges, "NW", uv, u_previous, k))
                    /dx)
                )
        S_term =(
                    Sfy*A.coef(xy, "S", uv)*(
                        (boundaryCellTraction.corner(edges, "SE", uv, u_previous, k) - boundaryCellTraction.corner(edges, "SW", uv, u_previous, k))
                        /dx)
                ) 
        E_term =(
                    Sfx*A.coef(xy, "E", uv)*(
                        (boundaryCellTraction.corner(edges, "NE", uv, u_previous, k) - boundaryCellTraction.corner(edges, "SE", uv, u_previous, k))
                        /dy)
                ) 
        W_term =(
                    Sfx*A.coef(xy, "W", uv)*(
                        (boundaryCellTraction.corner(edges, "NW", uv, u_previous, k) - boundaryCellTraction.corner(edges, "SW", uv, u_previous, k))
                        /dy)
                )

        for edge in edges:

            if (edge == "b") & (xy == "x") : S_term =  Sfy*tr_bottom_x  
            if (edge == "b") & (xy == "y") : S_term =  Sfy*tr_bottom_y  
            if (edge == "t") & (xy == "x") : N_term =  Sfy*tr_top_x  
            if (edge == "t") & (xy == "y") : N_term =  Sfy*tr_top_y  
            if (edge == "l") & (xy == "x") : W_term =  Sfx*tr_left_x  
            if (edge == "l") & (xy == "y") : W_term =  Sfx*tr_left_y  
            if (edge == "r") & (xy == "x") : E_term =  Sfx*tr_right_x  
            if (edge == "r") & (xy == "y") : E_term =  Sfx*tr_right_y        

        b_diffusion = (N_term + S_term + E_term + W_term)

        return b_diffusion
    
    def corner(edges, corner_placement, uv, u_previous, k):

        if uv == "u":
            uv_i = 0
        elif uv == "v":
            uv_i = 1

        disp = displacement(k, u_previous, uv_i)

        if corner_placement == "NE":
            corner =  (1/4)*(disp.P + disp.NE + disp.N + disp.E)
        if corner_placement == "SE":
            corner =  (1/4)*(disp.P + disp.SE + disp.S + disp.E)
        if corner_placement == "SW":
            corner =  (1/4)*(disp.P + disp.SW + disp.S + disp.W)
        if corner_placement == "NW":
            corner =  (1/4)*(disp.P + disp.NW + disp.N + disp.W)

        for edge in edges:
            if (edge == "b") & (corner_placement == "SE"):
                corner =  (1/2)*(disp.SE + disp.S)
            if (edge == "b") & (corner_placement == "SW"):
                corner =  (1/2)*(disp.SW + disp.S)

            if (edge == "t") & (corner_placement == "NE"):
                corner =  (1/2)*(disp.NE + disp.N)
            if (edge == "t") & (corner_placement == "NW"):
                corner =  (1/2)*(disp.NW + disp.N)

            if (edge == "l") & (corner_placement == "NW"):
                corner =  (1/2)*(disp.NW + disp.W)
            if (edge == "l") & (corner_placement == "SW"):
                corner =  (1/2)*(disp.SW + disp.W)

            if (edge == "r") & (corner_placement == "NE"):
                corner =  (1/2)*(disp.NE + disp.E)
            if (edge == "r") & (corner_placement == "SE"):
                corner =  (1/2)*(disp.SE + disp.E)
        
        return corner

# boundaryCellTraction.b_diff(["b", "l"], 14, "x", u_previous)
   

In [590]:
class boundaryCellDisplacement(A):

    def __init__(self, edges, xy):
        
        sw_S, sw_N, sw_W, sw_E = 1, 1, 1, 1

        for edge in edges:
            if edge == "b": sw_S = 2
            if edge == "t": sw_N = 2
            if edge == "l": sw_W = 2
            if edge == "r": sw_E = 2
        
        self.a_P = (rho*dx*dy/(dt**2)) + A(xy).a_N*sw_N + A(xy).a_S**sw_S + A(xy).a_E*sw_E + A(xy).a_W*sw_W
        
        self.a_N = A(xy).a_N*sw_N
        self.a_S = A(xy).a_S*sw_S
        self.a_E = A(xy).a_E*sw_E
        self.a_W = A(xy).a_W*sw_W

    def b_temp(u_old, u_old_old, k, xy):      
        return A.b_temp(u_old, u_old_old, k, xy)
    
    def b_diff(edges, k, xy, u_previous):

        if xy == "x":
            uv = "v"
        if xy == "y":
            uv = "u"
            
        N_term =(
                    Sfy*A.coef(xy, "N", uv)*(
                    (boundaryCellDisplacement.corner(edges, "NE", uv, u_previous, k) - boundaryCellDisplacement.corner(edges, "NW", uv, u_previous, k))
                    /dx)
                )
        S_term =(
                    Sfy*A.coef(xy, "S", uv)*(
                        (boundaryCellDisplacement.corner(edges, "SE", uv, u_previous, k) - boundaryCellDisplacement.corner(edges, "SW", uv, u_previous, k))
                        /dx)
                ) 
        E_term =(
                    Sfx*A.coef(xy, "E", uv)*(
                        (boundaryCellDisplacement.corner(edges, "NE", uv, u_previous, k) - boundaryCellDisplacement.corner(edges, "SE", uv, u_previous, k))
                        /dy)
                ) 
        W_term =(
                    Sfx*A.coef(xy, "W", uv)*(
                        (boundaryCellDisplacement.corner(edges, "NW", uv, u_previous, k) - boundaryCellDisplacement.corner(edges, "SW", uv, u_previous, k))
                        /dy)
                ) 

        b_diffusion = (N_term + S_term + E_term + W_term)

        return b_diffusion
    
    def corner(edges, corner_placement, uv, u_previous, k):

        if uv == "u":
            uv_i = 0
        elif uv == "v":
            uv_i = 1

        disp = displacement(k, u_previous, uv_i)

        if corner_placement == "NE":
            corner =  (1/4)*(disp.P + disp.NE + disp.N + disp.E)
        if corner_placement == "SE":
            corner =  (1/4)*(disp.P + disp.SE + disp.S + disp.E)
        if corner_placement == "SW":
            corner =  (1/4)*(disp.P + disp.SW + disp.S + disp.W)
        if corner_placement == "NW":
            corner =  (1/4)*(disp.P + disp.NW + disp.N + disp.W)

        for edge in edges:
            if (edge == "b") & (corner_placement == "SE"):
                corner =  (1/2)*(disp.SE + disp.S)
            if (edge == "b") & (corner_placement == "SW"):
                corner =  (1/2)*(disp.SW + disp.S)

            if (edge == "t") & (corner_placement == "NE"):
                corner =  (1/2)*(disp.NE + disp.N)
            if (edge == "t") & (corner_placement == "NW"):
                corner =  (1/2)*(disp.NW + disp.N)

            if (edge == "l") & (corner_placement == "NW"):
                corner =  (1/2)*(disp.NW + disp.W)
            if (edge == "l") & (corner_placement == "SW"):
                corner =  (1/2)*(disp.SW + disp.W)

            if (edge == "r") & (corner_placement == "NE"):
                corner =  (1/2)*(disp.NE + disp.E)
            if (edge == "r") & (corner_placement == "SE"):
                corner =  (1/2)*(disp.SE + disp.E)

        if len(edges) > 1:

            if (edges[0] == "b") & (edges[1] == "l") & (corner_placement == "SW"):
                corner = disp.SW 
            if (edges[0] == "b") & (edges[1] == "r") & (corner_placement == "SE"):
                corner = disp.SE
            if (edges[0] == "t") & (edges[1] == "l") & (corner_placement == "NW"):
                corner = disp.NW
            if (edges[0] == "t") & (edges[1] == "r") & (corner_placement == "NE"):
                corner = disp.NE

        
        return corner

# boundaryCellDisplacement.b_diff(["b", "l"], 14, "x", u_previous)
   

In [591]:
def displacement_cell_BCs(A_matrix, b_matrix, k, edges, xy, u_old, u_old_old, u_previous):

    A_matrix[k,k] = boundaryCellDisplacement(edges, xy).a_P
    #an
    A_matrix[k,index(k).n] = - boundaryCellDisplacement(edges, xy).a_N        
    #as
    A_matrix[k,index(k).s] = - boundaryCellDisplacement(edges, xy).a_S   
    #ae
    A_matrix[k, index(k).e] = - boundaryCellDisplacement(edges, xy).a_E
    #aw
    A_matrix[k, index(k).w] = - boundaryCellDisplacement(edges, xy).a_W

    b_matrix[k] =(
                boundaryCellDisplacement.b_temp(u_old, u_old_old, k, xy)
                +
                boundaryCellDisplacement.b_diff(edges, k, xy, u_previous)
            )   

    return A_matrix, b_matrix

def displacement_point_BCs(A_matrix, b_matrix, k, edges, xy):
    
    A_matrix[k,k] = 1
    b_matrix[k] = edge_U(edges, xy).BC

    return A_matrix, b_matrix


In [592]:
edges = ["b"]

edge_U(edges, "x").BC

-1e-06

In [593]:
## Time loop

!rm -r savedVariables
! mkdir savedVariables

for time in t:

    print("\nTime = " + str(time))
    
    # Set number of maximum iterations for convergence
    
    icorr = 1
    maxcorr = 100000

    # Make directory

    ! mkdir savedVariables/{time}
    
    while True:
    # for iteration in [1]:
            
        u_previous = u_new #store latest guess values

        A_x = A("x").createMatrix()
        
        for k in np.arange(0,(nx+2)*(ny+2)):   # k is the cell number

            xy = "x"

            U = displacement(k, u_previous, 0)
            V = displacement(k, u_previous, 1)

            #Bottom left corner coefficients
               
            if cell_index().bottom_left_corner(k):
                edges = ["b", "l"]

                #print(str(k) + ": Bottom left corner coefficients")

                #ap
                A_x, b_x = displacement_cell_BCs(A_x, b_x, k, edges, xy, u_old, u_old_old, u_previous)

            #Bottom right corner coefficients
                
            elif  cell_index().bottom_right_corner(k):
                edges = ["b", "r"]

                #print(str(k) + ": Bottom right corner coefficients")

                A_x, b_x = displacement_cell_BCs(A_x, b_x, k, edges, xy, u_old, u_old_old, u_previous)

            # Top Left Corner Coeficients
                
            elif  cell_index().top_left_corner(k):
                edges = ["t", "l"]

                #print(str(k) + ":Top Left corner coefficients")

                A_x, b_x = displacement_cell_BCs(A_x, b_x, k, edges, xy, u_old, u_old_old, u_previous)

                
            # Top right corner coefficients
                
            elif  cell_index().top_right_corner(k):   
                edges = ["t", "r"]

                #print(str(k) + ":Top right corner coefficients")

                A_x, b_x = displacement_cell_BCs(A_x, b_x, k, edges, xy, u_old, u_old_old, u_previous)

                
            # Center Bottom Boundaries

            elif  cell_index().center_bottom(k):
                edges = ["b"]

                #print(str(k) + ": Bottom Center Coefficents for Cell")

                A_x, b_x = displacement_cell_BCs(A_x, b_x, k, edges, xy, u_old, u_old_old, u_previous)
                               

            # Center Top Boundaries

            elif cell_index().center_top(k):
                edges = ["t"]

                #print(str(k) + ": Top Center Coefficents for Cell")
                A_x, b_x = displacement_cell_BCs(A_x, b_x, k, edges, xy, u_old, u_old_old, u_previous)

            
            # Center Left Boundaries

            elif  cell_index().center_left(k):
                edges = ["l"]

                #print(str(k) + ": Left Center Coefficents for Cell")

                A_x, b_x = displacement_cell_BCs(A_x, b_x, k, edges, xy, u_old, u_old_old, u_previous)


            # Center Right Boundaries

            elif  cell_index().center_right(k):
                edges = ["r"]

                #print(str(k) + ": Right Center Coefficents for Cell")

                A_x, b_x = displacement_cell_BCs(A_x, b_x, k, edges, xy, u_old, u_old_old, u_previous)


            #### Points on Boundary ####

            # Center Bottom

            elif boundary_point_index().center_bottom(k):
                edges = ["b"]

                # A_x[k,k] = 1
                # b_x[k] = edge_U(edges, xy).BC

                A_x, b_x = displacement_point_BCs(A_x, b_x, k, edges, xy)


            # Center Top

            elif boundary_point_index().center_top(k):
                edges = ["t"]

                A_x[k,k] = 1
                b_x[k] = edge_U(edges, xy).BC
                
            # Center Left

            elif boundary_point_index().center_left(k):
                edges = ["l"]

                A_x[k,k] = 1
                b_x[k] = edge_U(edges, xy).BC

            # Center Right

            elif boundary_point_index().center_right(k):
                edges = ["r"]

                A_x[k,k] = 1
                b_x[k] = edge_U(edges, xy).BC 

            # Bottom Left: Bottom

            elif boundary_point_index().bottom_left_bottom(k):
                edges = ["b", "l"]

                A_x[k,k] = 1
                b_x[k] = edge_U(edges, xy).BC

            # Bottom Left: Left

            elif boundary_point_index().bottom_left_left(k):
                edges = ["l", "b"]

                A_x[k,k] = 1
                b_x[k] = edge_U(edges, xy).BC
            
            # Bottom Right: Right

            elif boundary_point_index().bottom_right_right(k):
                edges = ["r", "b"]

                A_x[k,k] = 1
                b_x[k] = edge_U(edges, xy).BC

            # Bottom Right: Bottom

            elif boundary_point_index().bottom_right_bottom(k):
                edges = ["b", "r"]

                A_x[k,k] = 1
                b_x[k] = edge_U(edges, xy).BC

            # Top Left: Left

            elif boundary_point_index().top_left_left(k):
                edges = ["l", "t"]

                A_x[k,k] = 1
                b_x[k] = edge_U(edges, xy).BC

            # Top Left: Top

            elif boundary_point_index().top_left_top(k):
                edges = ["t", "l"]

                A_x[k,k] = 1
                b_x[k] = edge_U(edges, xy).BC

            # Top Right: Right

            elif boundary_point_index().top_right_right(k):
                edges = ["r", "t"]

                A_x[k,k] = 1
                b_x[k] = edge_U(edges, xy).BC

            # Top Right: Top
            
            elif boundary_point_index().top_right_top(k):
                edges = ["t", "r"]

                A_x[k,k] = 1
                b_x[k] = edge_U(edges, xy).BC
                        
            # Corner Points

            elif boundary_point_index().bottom_left_corner(k):

                #print(str(k) + ": Bottom Left Corner Point on Boundary")

                A_x[k,k] = 4

                A_x[k,index(k).n] = 2 
                A_x[k,index(k).e] = 2

                A_x[k,index(k).ne] = -15

                A_x[k,index(k).dir(["n", "n"])] = 2 
                A_x[k,index(k).dir(["e", "e"])] = 2

                A_x[k,index(k).dir(["n", "ne"])] = 1
                A_x[k,index(k).dir(["ne", "ne"])] = 1
                A_x[k,index(k).dir(["e", "ne"])] = 1

                b_x[k] = 0

                # edges = ["b"]

                # A_x[k,k] = 1
                # b_x[k] = edge_U(edges, xy).BC

            elif boundary_point_index().bottom_right_corner(k):
                #print(str(k) + ": Bottom Right Corner Point on Boundary")
                A_x[k,k] = 4

                A_x[k,index(k).n] = 2 
                A_x[k,index(k).w] = 2

                A_x[k,index(k).nw] = -15

                A_x[k,index(k).dir(["n", "n"])] = 2 
                A_x[k,index(k).dir(["w", "w"])] = 2

                A_x[k,index(k).dir(["n", "nw"])] = 1
                A_x[k,index(k).dir(["nw", "nw"])] = 1
                A_x[k,index(k).dir(["w", "nw"])] = 1

                # edges = ["b"]

                # A_x[k,k] = 1
                # b_x[k] = edge_U(edges, xy).BC

            elif boundary_point_index().top_left_corner(k):
                #print(str(k) + ": Top Left Corner Point on Boundary")
                A_x[k,k] = 4

                A_x[k,index(k).s] = 2 
                A_x[k,index(k).e] = 2

                A_x[k,index(k).se] = -15

                A_x[k,index(k).dir(["s", "s"])] = 2 
                A_x[k,index(k).dir(["e", "e"])] = 2

                A_x[k,index(k).dir(["s", "se"])] = 1
                A_x[k,index(k).dir(["se", "se"])] = 1
                A_x[k,index(k).dir(["e", "se"])] = 1

                b_x[k] = 0
                # edges = ["t"]

                # A_x[k,k] = 1
                # b_x[k] = edge_U(edges, xy).BC

            elif boundary_point_index().top_right_corner(k):
                #print(str(k) + ": Top Right Corner Point on Boundary")
                A_x[k,k] = 4

                A_x[k,index(k).s] = 2 
                A_x[k,index(k).w] = 2

                A_x[k,index(k).sw] = -15

                A_x[k,index(k).dir(["s", "s"])] = 2 
                A_x[k,index(k).dir(["w", "w"])] = 2

                A_x[k,index(k).dir(["s", "sw"])] = 1
                A_x[k,index(k).dir(["sw", "sw"])] = 1
                A_x[k,index(k).dir(["w", "sw"])] = 1

                b_x[k] = 0
                # edges = ["t"]

                # A_x[k,k] = 1
                # b_x[k] = edge_U(edges, xy).BC

                
            else:

                # print(str(k) + ": NO BC Cell")

                b_x[k] = (
                            A.b_temp(u_old, u_old_old, k, "x")
                            +
                            A.b_diff(k, "x", u_previous)
                        )

        # saveArray(str(time) + "/A_x_" + str(icorr) , A_x.todense())   
        # saveArray(str(time) + "/b_x_" + str(icorr), b_x)  

        ## Solve the x-equation
        
        Ux = spsolve(A_x, b_x)
        Ux = np.array(Ux)

        # #print(Ux)

        saveArray(str(time) + "/Ux_" + str(icorr), Ux)
        
        ## Prepare matrix A and b vector for the y-equation
        
        A_y = A("y").createMatrix()
        
        for k in np.arange(0,(nx+2)*(ny+2)):   # j is the cell number

            xy = "y"


            U = displacement(k, u_previous, 0)
            V = displacement(k, u_previous, 1)
 
            #Bottom left corner coefficients
            
            if cell_index().bottom_left_corner(k):
                edges = ["b", "l"]

                #print(str(k) + ": Bottom left corner coefficients")

                A_y, b_y = displacement_cell_BCs(A_y, b_y, k, edges, xy, u_old, u_old_old, u_previous)

            
            #Bottom right corner coefficients
                
            elif  cell_index().bottom_right_corner(k):
                edges = ["b", "r"]

                #print(str(k) + ": Bottom right corner coefficients")

                A_y, b_y = displacement_cell_BCs(A_y, b_y, k, edges, xy, u_old, u_old_old, u_previous)

                     
            #Top left corner coefficients
                
            elif  cell_index().top_left_corner(k):
                edges = ["t", "l"]

                #print(str(k) + ":Top Left corner coefficients")

                A_y, b_y = displacement_cell_BCs(A_y, b_y, k, edges, xy, u_old, u_old_old, u_previous)

                
            #Top right corner coefficients
                
            elif  cell_index().top_right_corner(k):   
                edges = ["t", "r"]

                #print(str(k) + ":Top right corner coefficients")
                
                A_y, b_y = displacement_cell_BCs(A_y, b_y, k, edges, xy, u_old, u_old_old, u_previous)

            
            # Center Bottom Boundaries

            elif  cell_index().center_bottom(k):
                edges = ["b"]

                #print(str(k) + ": Bottom Center Coefficents for Cell")
                A_y, b_y = displacement_cell_BCs(A_y, b_y, k, edges, xy, u_old, u_old_old, u_previous)


            # Center Top Boundaries

            elif cell_index().center_top(k):
                edges = ["t"]

                #print(str(k) + ": Top Center Coefficents for Cell")

                A_y, b_y = displacement_cell_BCs(A_y, b_y, k, edges, xy, u_old, u_old_old, u_previous)


            # Center Left Boundaries

            elif  cell_index().center_left(k):

                edges = ["l"]

                #print(str(k) + ": Left Center Coefficents for Cell")

                A_y, b_y = displacement_cell_BCs(A_y, b_y, k, edges, xy, u_old, u_old_old, u_previous)


            # Center Right Boundaries

            elif  cell_index().center_right(k):
                edges = ["r"]

                #print(str(k) + ": Right Center Coefficents for Cell")

                A_y, b_y = displacement_cell_BCs(A_y, b_y, k, edges, xy, u_old, u_old_old, u_previous)



            #### Points on Boundary ####

            # Center Bottom

            elif boundary_point_index().center_bottom(k):
                edges = ["b"]

                A_y[k,k] = 1
                b_y[k] = edge_U(edges, xy).BC

            # Center Top

            elif boundary_point_index().center_top(k):
                edges = ["t"]

                A_y[k,k] = 1
                b_y[k] = edge_U(edges, xy).BC
                
            # Center Left

            elif boundary_point_index().center_left(k):
                edges = ["l"]

                A_y[k,k] = 1
                b_y[k] = edge_U(edges, xy).BC

            # Center Right

            elif boundary_point_index().center_right(k):
                edges = ["r"]

                A_y[k,k] = 1
                b_y[k] = edge_U(edges, xy).BC

            # Bottom Left: Bottom

            elif boundary_point_index().bottom_left_bottom(k):
                edges = ["b", "l"]

                A_y[k,k] = 1
                b_y[k] = edge_U(edges, xy).BC

            elif boundary_point_index().bottom_left_left(k):
                edges = ["l", "b"]

                A_y[k,k] = 1
                b_y[k] = edge_U(edges, xy).BC
            
            # Bottom Right: Right

            elif boundary_point_index().bottom_right_right(k):
                edges = ["r", "b"]

                A_y[k,k] = 1
                b_y[k] = edge_U(edges, xy).BC

            # Bottom Right: Bottom

            elif boundary_point_index().bottom_right_bottom(k):
                edges = ["b", "r"]

                A_y[k,k] = 1
                b_y[k] = edge_U(edges, xy).BC

            # Top Left: Left

            elif boundary_point_index().top_left_left(k):
                edges = ["l", "t"]

                A_y[k,k] = 1
                b_y[k] = edge_U(edges, xy).BC

            # Top Left: Top

            elif boundary_point_index().top_left_top(k):
                edges = ["t", "l"]

                A_y[k,k] = 1
                b_y[k] = edge_U(edges, xy).BC

            # Top Right: Right

            elif boundary_point_index().top_right_right(k):
                edges = ["r", "t"]

                A_y[k,k] = 1
                b_y[k] = edge_U(edges, xy).BC

            # Top Right: Top
            
            elif boundary_point_index().top_right_top(k):
                edges = ["t", "r"]

                A_y[k,k] = 1
                b_y[k] = edge_U(edges, xy).BC

            # # Corner Points
            elif boundary_point_index().bottom_left_corner(k):

                #print(str(k) + ": Bottom Left Corner Point on Boundary")

                A_y[k,k] = 4

                A_y[k,index(k).n] = 2 
                A_y[k,index(k).e] = 2

                A_y[k,index(k).ne] = -15

                A_y[k,index(k).dir(["n", "n"])] = 2 
                A_y[k,index(k).dir(["e", "e"])] = 2

                A_y[k,index(k).dir(["n", "ne"])] = 1
                A_y[k,index(k).dir(["ne", "ne"])] = 1
                A_y[k,index(k).dir(["e", "ne"])] = 1

                b_y[k] = 0
                b_x[k] = 0
                # edges = ["b"]

                # A_y[k,k] = 1
                # b_y[k] = edge_U(edges, xy).BC

            elif boundary_point_index().bottom_right_corner(k):
                #print(str(k) + ": Bottom Right Corner Point on Boundary")
                A_y[k,k] = 4

                A_y[k,index(k).n] = 2 
                A_y[k,index(k).w] = 2

                A_y[k,index(k).nw] = -15

                A_y[k,index(k).dir(["n", "n"])] = 2 
                A_y[k,index(k).dir(["w", "w"])] = 2

                A_y[k,index(k).dir(["n", "nw"])] = 1
                A_y[k,index(k).dir(["nw", "nw"])] = 1
                A_y[k,index(k).dir(["w", "nw"])] = 1

                b_y[k] = 0
                # edges = ["b"]

                # A_y[k,k] = 1
                # b_y[k] = edge_U(edges, xy).BC


            elif boundary_point_index().top_left_corner(k):
                #print(str(k) + ": Top Left Corner Point on Boundary")
                A_y[k,k] = 4

                A_y[k,index(k).s] = 2 
                A_y[k,index(k).e] = 2

                A_y[k,index(k).se] = -15

                A_y[k,index(k).dir(["s", "s"])] = 2 
                A_y[k,index(k).dir(["e", "e"])] = 2

                A_y[k,index(k).dir(["s", "se"])] = 1
                A_y[k,index(k).dir(["se", "se"])] = 1
                A_y[k,index(k).dir(["e", "se"])] = 1

                b_y[k] = 0
                # edges = ["t"]

                # A_y[k,k] = 1
                # b_y[k] = edge_U(edges, xy).BC


            elif boundary_point_index().top_right_corner(k):
                #print(str(k) + ": Top Right Corner Point on Boundary")
                A_y[k,k] = 4

                A_y[k,index(k).s] = 2 
                A_y[k,index(k).w] = 2

                A_y[k,index(k).sw] = -15

                A_y[k,index(k).dir(["s", "s"])] = 2 
                A_y[k,index(k).dir(["w", "w"])] = 2

                A_y[k,index(k).dir(["s", "sw"])] = 1
                A_y[k,index(k).dir(["sw", "sw"])] = 1
                A_y[k,index(k).dir(["w", "sw"])] = 1

                b_y[k] = 0
                # edges = ["t"]

                # A_y[k,k] = 1
                # b_y[k] = edge_U(edges, xy).BC
            

            else:

                #print(str(k) + ": NO BC Cell")

                b_y[k] = (
                            A.b_temp(u_old, u_old_old, k, "y")
                            +
                            A.b_diff(k, "y", u_previous)
                        )
        # saveArray(str(time) + "/A_y_" + str(icorr) , A_y)
        # saveArray(str(time) + "/b_y_" + str(icorr), b_y)       
                
        ## Solve the y-equation
        Uy = spsolve(A_y, b_y)
        Uy = np.array(Uy)

        # saveArray(str(time) + "/Uy_" + str(icorr), Uy)
        
        # ## Update U.New with new x and y displacements

        u_new1 = np.vstack((Ux, Uy)).T
        # saveArray (str(time) + "/U_new1_" + str(icorr), u_new1)    
        # saveArray (str(time) + "/u_previous_" + str(icorr), u_previous)

        # # U.New[:,0] = Ux
        # # U.New[:,1] = Uy
        
        ## Calculate the residual of each iteration
        
        normFactor = np.max(u_new1)
        residual = math.sqrt(np.mean((u_new1 - u_previous)**2))/normFactor
        tolerance = 1e-6
        u_new = u_new1
    
        ##print out residual of each iteration
        print("icorr = {:01d}, Residual = {:.20f}, normFactor = {:.20f}".format(icorr, residual, normFactor))
             
        ## Convergence check
        
        if residual < tolerance:
            
            break
            
        elif icorr > maxcorr:
            
            break            
        
        icorr = icorr + 1
    
    ## Update variables
    
    u_old_old = u_old
    u_old = u_new1
    
    # ## Add the boundary conditions to the solution matrices
    u_reshape_x = u_new[:,0].reshape(nx+2, ny+2)
    u_reshape_y = u_new[:,1].reshape(nx+2, ny+2)
    
    # #Boundary conditions for u matrix
    U = np.zeros((nx+2,ny+2))
    V = np.zeros((nx+2,ny+2))
 
    U[:, :] = u_reshape_x
    V[:, :] = u_reshape_y 

    #Modulus of displacement

    disp_mod = np.sqrt((U**2) + (V**2))
    disp_mod_converted = disp_mod*1e6 #Convert displacement to micrometers
    
#     # ## Plot the results

    import plotly.graph_objects as go

    disp_mod = np.sqrt((U**2) + (V**2))*1e-10

    dispx, dispy = np.meshgrid(x, y)
    fig = go.Figure(data=[go.Surface(z=disp_mod_converted, x=dispx, y=dispy)])

    
    
fig.show()   

print("U graph")
fig = go.Figure(data=[go.Surface(z=U, x=dispx, y=dispy)])

fig.show()   

print("V graph")
fig = go.Figure(data=[go.Surface(z=V, x=dispx, y=dispy)])

fig.show()   






Time = 0.0
icorr = 1, Residual = 0.33249823757863083973, normFactor = 0.00000229857332906039
icorr = 2, Residual = 0.05792819909369520404, normFactor = 0.00000227816643218719
icorr = 3, Residual = 0.03009851397733159101, normFactor = 0.00000322679113148762
icorr = 4, Residual = 0.01962805868641278456, normFactor = 0.00000393383778341544
icorr = 5, Residual = 0.01353180207992573693, normFactor = 0.00000453498649230826
icorr = 6, Residual = 0.00980955481630328870, normFactor = 0.00000498382384337414
icorr = 7, Residual = 0.00725539400979238400, normFactor = 0.00000536483913915077
icorr = 8, Residual = 0.00548681083829498026, normFactor = 0.00000564946060229507
icorr = 9, Residual = 0.00419032454852893109, normFactor = 0.00000589100708652398
icorr = 10, Residual = 0.00323752276949098965, normFactor = 0.00000607147109285290
icorr = 11, Residual = 0.00251493158955878177, normFactor = 0.00000622460419533171
icorr = 12, Residual = 0.00196643225876660076, normFactor = 0.00000633902572325843
i

U graph


V graph


In [594]:
Ux[0] - Uy[0]

1.4190920165394087e-07